In [ ]:
def get_bin_pos2(bin_filename):
    '''this was an attempt to improve the speed of get_bin_pos, it is slower though'''
    byte_count = os.path.getsize(bin_filename)
    bytes_per_iteration = 432
    iteration_count = int(byte_count/bytes_per_iteration)  
    sample_count = iteration_count * 192  # each iteration has 192 samples (64*3)
    
    DaqFs = 48000
    duration = iteration_count * 3 / DaqFs
    duration = np.ceil(duration)
    
    #Reading the Data
    
    header_byte_len = 32
    data_byte_len = 384
    trailer_byte_len = 16
    
    byte_skip = data_byte_len +  trailer_byte_len # 384 ephys data bytes + 16 trailer bytes
    
    pos_Fs = 50
    n_samples = int(duration*pos_Fs)
    
    #post = np.zeros((n_samples,1))
    #posx = np.zeros_like(post)
    #posy = np.zeros_like(post)
    
    position_data = b''
    time_data = b''
    with open(bin_filename, 'rb') as f:
        #open the bin file
        #f.seek(12)
        
        for i in range(iteration_count):
            byte_id = f.read(4)

            if 'ADU2' in str(byte_id):
                f.seek(8, 1)  # moves the current position by 10 bytes
                data = f.read(20)
                
                #time_data = data[:4]
                #time_data = np.asarray(struct.unpack('i', time_data))
                
                time_data += data[:4]
                position_data += data[4:]
                
                #print(position_data)
                #print(len(position_data))
                    
                f.seek(byte_skip,1)
                
            else:
                f.seek(bytes_per_iteration-4, 1)
    
    # calculate the number of position samples
    pos_sample_num = int(len(position_data)/16) # 16 bytes per sample
    
    word_count = int(pos_sample_num * 8)  # each position sample contains 8 word values
    
    position_data = np.asarray(struct.unpack('%dh'% word_count, position_data))
    position_data = np.reshape(position_data, (pos_sample_num, 8))
    
    time_data = np.asarray(struct.unpack('%di' % pos_sample_num, time_data))
    
    post = time_data
    posx = position_data[:,0]
    posy = position_data[:,1]
    
    # converting the time value from a frame count to a time value
    timebase = 50
    post = np.multiply(post, (1 / int(timebase)))  # converting the frame number from Axona to the time value

    # find the 1023 values, which indicates the light wasn't tracked

    invalid_positions = np.where((posx == 1023) | (posy == 1023))[0]
    
    posx = posx.astype(float)
    posy 
    
    try:
        posx[invalid_positions] = np.nan
        posy[invalid_positions] = np.nan   
        
    except ValueError:
        posx = posx.astype(float)
        posy = posy.astype(float) 
        posx[invalid_positions] = np.nan
        posy[invalid_positions] = np.nan  
    
    #print(pos_sample_num)
    
    return posx.reshape((len(posx),1)), posy.reshape((len(posy),1)), post.reshape((len(post),1))